# Tool Calling with Agents and using Threads

In this sample we will be creating agents and use a tool (function) to provide answers to questions. We will also be creating a thread and then doing a follow-up with the agent to ask for more information.

You should have the following two environment variables already created and initialized:

* `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI project endpoint
* `AZURE_AI_MODEL_DEPLOYMENT_NAME`: The name of your model deployment

In [1]:
import asyncio
import os
from random import randint
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

load_dotenv()

project_endpoint = os.environ.get('AZURE_AI_PROJECT_ENDPOINT')
model_name = os.environ.get('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print(f"Project endpoint: {project_endpoint}")
print(f"Deployment name: {model_name}")

Project endpoint: https://ai-service-wus3-shared.services.ai.azure.com/api/projects/agentic-project
Deployment name: gpt-4o


In [2]:
# 🎲 Tool Function: Random Destination Generator
# This function will be available to the agent as a tool
# The agent can call this function to get random vacation destinations
def get_random_destination() -> str:
    """Get a random vacation destination.
    
    Returns:
        str: A randomly selected destination from our predefined list
    """
    # List of popular vacation destinations around the world
    destinations = [
        "Rome", "Italy",
        "Barcelona, Spain",
        "Paris, France", 
        "Berlin, Germany",
        "Tokyo, Japan",
        "Sydney, Australia",
        "New York, USA",
        "Cairo, Egypt",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bali, Indonesia"
    ]
    # Return a random destination from the list
    return destinations[randint(0, len(destinations) - 1)]

In [3]:
async def main():
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(async_credential=credential)
    agent = ChatAgent(chat_client, name="Vacation Planner",
                instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations.",
                tools=[get_random_destination]
            )
                                                             
    thread = agent.get_new_thread()
    result = await agent.run("Please plan a trip for me", thread=thread)
    print(result.text)

    second_result = await agent.run("I do not like that destination. Please plan another vacation for me", thread=thread)
    print(second_result.text)

    await chat_client.close()
    

In [4]:
await main()

[2025-10-07 20:41:10 - c:\Users\aljafry\repos\AgentFramework\azure-ai-learn\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-10-07 20:41:19 - c:\Users\aljafry\repos\AgentFramework\azure-ai-learn\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Your randomly selected destination is Barcelona, Spain! Let me help you plan your trip:

### Trip Itinerary for Barcelona, Spain

#### **Day 1: Arrival and Exploration**
- **Arrival**
  - Fly into Barcelona-El Prat Airport. Arrange airport transfer to your accommodation.
  - Check in to a centrally located hotel near Plaça de Catalunya or Las Ramblas.
- **Evening**
  - Stroll along Las Ramblas and explore Mercat de Sant Josep de la Boqueria (local market).
  - Enjoy a traditional Spanish dinner with paella and sangria.

#### **Day 2: Gaudi Extravaganza**
- **Morning**
  - Visit the iconic Sagrada Família designed by Antoni Gaudí. Pre-book tickets to skip the line.
- **Afternoon**
  - Explore Park Güell, a colorful, whimsical park full of Gaudí's architecture and stunning views of the city.
- **Evening**
  - Watch the Magic Fountain Show at Montjuïc. It’s a spectacular show combining lights, music, and water.

#### **Day 3: Gothic Quarter and History**
- **Morning**
  - Walk through the

[2025-10-07 20:41:22 - c:\Users\aljafry\repos\AgentFramework\azure-ai-learn\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Your new randomly selected destination is New York, USA! Here's a suggested plan for your trip:

### Trip Itinerary for New York City

#### **Day 1: Arrival and Evening in Times Square**
- **Arrival**
  - Fly into JFK, LaGuardia, or Newark Airport. Arrange airport transfer into Manhattan.
  - Check into a hotel in Midtown Manhattan near Times Square or Central Park.
- **Evening**
  - Explore Times Square with its glowing billboards, shops, and street performances.
  - Dine at a classic New York City restaurant or enjoy a slice of famous New York pizza.

#### **Day 2: Iconic Attractions and Central Park**
- **Morning**
  - Start with a visit to the Statue of Liberty and Ellis Island. Book a ferry ahead of time.
- **Afternoon**
  - Walk through Wall Street and see the New York Stock Exchange and the Charging Bull.
  - Visit the 9/11 Memorial and Museum for a powerful and emotional experience.
- **Evening**
  - Stroll through Central Park. Visit Bethesda Terrace or rent a rowboat at the L